In [2]:
import os, glob
import numpy as np
import pandas as pd
from scipy import stats
from datetime import date, time, datetime
from datetime import timedelta
from sklearn.model_selection import train_test_split
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.utils import np_utils
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,  accuracy_score


In [4]:
w_list= glob.glob("../result/merge_weather/*.csv")
p_list = glob.glob("../result/merger_by_date//*.csv")
weather_price_pair = [("Anthurium","467480"), ("Chrysanthemum", "C0F9L0" ),
                      ("Eustoma", "C0F9L0"),("OrientalLily", "C0X060"),  ("Rose", "C0X060") ]
for flower,st in weather_price_pair:
    print(flower,"\t",st)

Anthurium 	 467480
Chrysanthemum 	 C0F9L0
Eustoma 	 C0F9L0
OrientalLily 	 C0X060
Rose 	 C0X060


In [13]:
def pick_flower_station(flower,st):
    fn_weather = glob.glob("../result/merge_weather/*"+ st +"*.csv")[0]
    fn_price = glob.glob("../result/merger_by_date//"+ flower +"*.csv")[0]
    return fn_weather,fn_price


for flower,st in weather_price_pair:
    fn_weather,fn_price = pick_flower_station(flower,st)
    print(fn_weather,fn_price)

../result/merge_weather\467480_嘉義.csv ../result/merger_by_date\Anthurium.csv
../result/merge_weather\C0F9L0_后里.csv ../result/merger_by_date\Chrysanthemum.csv
../result/merge_weather\C0F9L0_后里.csv ../result/merger_by_date\Eustoma.csv
../result/merge_weather\C0X060_下營.csv ../result/merger_by_date\OrientalLily.csv
../result/merge_weather\C0X060_下營.csv ../result/merger_by_date\Rose.csv


In [42]:
# df_weather.join(df_flower.set_index("date"), on= "Date")
df_join = df_flower.join(df_weather.set_index("Date"), on= "date").join(df_lunar.set_index("date"), on= "date")

In [146]:
weather_price_pair = [("Anthurium","467480"), ("Chrysanthemum", "C0F9L0" ),
                      ("Eustoma", "C0F9L0"),("OrientalLily", "C0X060"),  ("Rose", "C0F9L0") ]

flower,st = weather_price_pair[4]
fn_weather,fn_price = pick_flower_station(flower,st)

In [149]:
def join_weather_price(fn_price,fn_weather):
    df_flower = pd.read_csv(fn_price)
    df_weather = pd.read_csv(fn_weather)
    df_lunar = pd.read_csv("../lunar_done_new.csv")
    # diff1 刪掉 2014-1-1  diff2 刪掉 2014-1-2
    df_join = df_flower.join(df_weather.set_index("Date"), on= "date").join(df_lunar.set_index("date"), on= "date")
    price = df_join["price"]
    pc_c = price.pct_change()
    df_join["per_change"] = pc_c
    df_join["diff1"] = price.diff()
    df_join = df_join[df_join.date != "2014-01-01"]
    df_join = df_join.reset_index(drop=True)
    df_join["diff2"] = df_join["diff1"].diff()
    df_join = df_join[df_join.date != "2014-01-02"]
    df_join = df_join.reset_index(drop=True)
    df_join["station"] = st
    # re order columns
    order_c = [
                'date', 'market', 'station',  'price', 'count', 'per_change', 'diff1', 'diff2','year',
                'week', 'flower', 'Temperature', 'T.Max', 'T.Min',
                'Precp', 'RH', 'StnPres', 'WS', 'WSGust',
                'g_marry', 'g_pray', 'g_funeral', 'g_business', 'b_marry',
                'b_pray', 'b_funeral', 'b_business','spring',
                'valetine_west', 'mother', 'graduation', 'ghost', 'valetine_east',
                ]
    df_done = df_join[order_c]
    fn_save = "../result/join_all/"+flower + "_" + st + "_join_all.csv"
    df_done.to_csv(fn_save,index=False, encoding="utf-8")
    print(fn_save, "saved")

In [166]:
def out_join_weather_price(fn_price,fn_weather):
    df_flower = pd.read_csv(fn_price)
    df_weather = pd.read_csv(fn_weather)
    df_lunar = pd.read_csv("../lunar_done_new.csv")
    # diff1 刪掉 2014-1-1  diff2 刪掉 2014-1-2
    df_join = df_flower.join(df_weather.set_index("Date"), on= "date", how="outer")\
                       .join(df_lunar.set_index("date"), on= "date", how="outer")
    price = df_join["price"]
    pc_c = price.pct_change()
    df_join["per_change"] = pc_c
    df_join["diff1"] = price.diff()
    df_join = df_join[df_join.date != "2014-01-01"]
    df_join = df_join.reset_index(drop=True)
    df_join["diff2"] = df_join["diff1"].diff()
    df_join = df_join[df_join.date != "2014-01-02"]
    df_join = df_join.reset_index(drop=True)
    df_join["station"] = st
    # re order columns
    order_c = [
                'date', 'market', 'station',  'price', 'count', 'per_change', 'diff1', 'diff2','year',
                'week', 'flower', 'Temperature', 'T.Max', 'T.Min',
                'Precp', 'RH', 'StnPres', 'WS', 'WSGust',
                'g_marry', 'g_pray', 'g_funeral', 'g_business', 'b_marry',
                'b_pray', 'b_funeral', 'b_business','spring',
                'valetine_west', 'mother', 'graduation', 'ghost', 'valetine_east',
                ]
    df_done = df_join[order_c]
    fn_save = "../result/join_all/out_join"+flower + "_" + st + ".csv"
    df_done.to_csv(fn_save,index=False, encoding="utf-8")
    print(fn_save, "saved")

In [167]:
weather_price_pair = [("Anthurium","467480"), ("Chrysanthemum", "C0F9L0" ),
                      ("Eustoma", "C0F9L0"),("OrientalLily", "C0X060"),  ("Rose", "C0F9L0") ]

# flower,st = weather_price_pair[4]
for flower,st in weather_price_pair:
    fn_weather,fn_price = pick_flower_station(flower,st)
    out_join_weather_price(fn_price,fn_weather)

../result/join_all/out_joinAnthurium_467480.csv saved
../result/join_all/out_joinChrysanthemum_C0F9L0.csv saved
../result/join_all/out_joinEustoma_C0F9L0.csv saved
../result/join_all/out_joinOrientalLily_C0X060.csv saved
../result/join_all/out_joinRose_C0F9L0.csv saved


In [168]:
df = pd.read_csv("../result/join_all/out_joinAnthurium_467480.csv")

In [175]:
df = df.sort_values(by=["date"]).reset_index(drop=True)
df.to_csv("test.csv", index=False)

In [ ]:
# "Anthurium" "467480"     shift:(14,1,55~63%), (27,8,46~65%) (19,15,55~72%)
# "Chrysanthemum" "C0F9L0" shift:(19,1,51~58%), (16,8,40~59%) (16,15,33~50%)
# "Eustoma" "C0F9L0"       shift:(3,1,67~73%) (16,8,49~57%) (12,15,51~63%)  epochs [10,]
# "OrientalLily" "C0X060"  shift:(17,1,77~79%) (22,8,66~75%) (14,15,43~64%) epochs [ , 12, ]
# "Rose" "C0X060"          shift:(7,1,47~64%) (18,8,42~64%) (18,15,55~69%)